# PySpark 를 이용한 Steam 게임 추천 알고리즘

### 임포팅

In [ ]:
# written in python3
# pyspark version 3.0.1

!pip install pyspark
import os
import pyspark as spark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, HiveContext, Row
from pyspark.sql.functions import desc
from pyspark.mllib.recommendation import ALS
import json

     |████████████████████████████████| 204.2MB 76kB/s 
     |████████████████████████████████| 204kB 33.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=2b62caad3d285b457bb02ae31024e44bbf1296cae0c5246e24c0204132438cd3
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
# 데이터 경로 임포팅
# 자기가 돌릴 때는 데이터 셋과 결과물 세트의 경로를 조정하세요
# 그대로 사용하고 싶으면 colab을 googledrive 와 연동하고, drive 내에 Colab Notebooks/ 안에 ipynb파일을 저장하고, Colab Notebooks/dataset/에  데이터셋을 집어넣으면 작동합니다.
# 하기 폴더가 존재하지 않으면 에러 생성함

game_detail = '/content/drive/MyDrive/Colab Notebooks/dataset/game_detail_v2.json'
user_owned_games = '/content/drive/MyDrive/Colab Notebooks/dataset/user_owned_games.json'
user_friend_list = '/content/drive/MyDrive/Colab Notebooks/dataset/user_friend_list.json'
user_recent_games = '/content/drive/MyDrive/Colab Notebooks/dataset/user_recently_played_games.json'
user_idx = '/content/drive/MyDrive/Colab Notebooks/dataset/user_idx.json'
recommended = '/content/drive/MyDrive/Colab Notebooks/dataset/recommended.json'
final_recommended = '/content/drive/MyDrive/Colab Notebooks/dataset/final_recommended'

## PySpark

PySpark 는 Spark를 python 에서 사용할 수 있게 해주는 라이브러리입니다.

Spark는 빅데이터 분석을 위한 병렬 분산 처리 플랫폼입니다.



### PySpark 세션 초기화

In [ ]:
# pyspark 세션 빌딩
# Hive Support = Enable

sc = SparkSession.builder.appName("spark-recommender").enableHiveSupport().getOrCreate()

### 게임 디테일 처리

In [ ]:
# 게임 디테일 df 변경

df_game = sc.read.json(game_detail)
df_game.printSchema()

root
 |-- about_the_game: string (nullable = true)
 |-- achievements: struct (nullable = true)
 |    |-- highlighted: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- background: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |-- content_descriptors: struct (nullable = true)
 |    |-- ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- notes: string (nullable = true)
 |-- controller_support: string (nullable = true)
 |-- demos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appid: long (nullable = true)
 |    |    |-- description: string (nullable = true)
 |-- detailed_descript

In [ ]:
# 게임 디테일 유효성 검사

df_game.registerTempTable("temp_game_detail")
df_valid_game = sc.sql("SELECT * FROM temp_game_detail")
df_valid_game.registerTempTable("game_detail")
df_valid_game.show(1)

+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+------------+--------------------+--------------------+----+----------+-----------------------+--------------------+--------------------+-------+--------------------+------------------+----------------+----------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+----------+---------------+--------------------+------------+-------+--------------------+--------------------+-----------+--------------------+--------------------+----+--------------------+
|      about_the_game|        achievements|          background|          categories|content_descriptors|controller_support|       demos|detailed_description|          developers| dlc|drm_notice|ext_user_account_notice|              genres|        header_image|is_free|        legal_notice|linux_requirements|mac_requirements|meta

### 유저 소유 게임 처리

In [ ]:
# 유저 소유 게임 입력

df_user_owned_games = sc.read.json(user_owned_games)
df_user_owned_games.printSchema()
df_user_owned_games.registerTempTable("user_owned_games")

root
 |-- game_count: string (nullable = true)
 |-- games: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appid: long (nullable = true)
 |    |    |-- playtime_2weeks: long (nullable = true)
 |    |    |-- playtime_forever: long (nullable = true)
 |    |    |-- playtime_linux_forever: long (nullable = true)
 |    |    |-- playtime_mac_forever: long (nullable = true)
 |    |    |-- playtime_windows_forever: long (nullable = true)
 |-- steamid: string (nullable = true)



In [ ]:
# 사람들이 가장 많이 플레이 한 10개의 게임 추천(콜드 스타트 문제 해결)
df_global_popular_games = \
sc.sql("SELECT b.game_id, SUM(b.playtime_forever) AS play_time FROM \
                (SELECT played_games['appid'] AS game_id, played_games['playtime_forever'] AS playtime_forever \
                FROM (SELECT EXPLODE(games) AS played_games FROM user_owned_games) a) b \
                GROUP BY game_id ORDER BY play_time DESC LIMIT 10")
df_global_popular_games.registerTempTable('popular_games')

# rank = 유저 전부 합친 플레이타임
df_global_popular_games = sc.sql("SELECT b.name AS name, a.play_time AS rank, b.steam_appid, b.header_image FROM \
                                    popular_games a, game_detail b WHERE a.game_id = b.steam_appid ORDER BY rank DESC")
df_global_popular_games.show()

+--------------------+---------+-----------+--------------------+
|                name|     rank|steam_appid|        header_image|
+--------------------+---------+-----------+--------------------+
|Counter-Strike: G...|136557735|        730|https://steamcdn-...|
|         Garry's Mod| 68785425|       4000|https://steamcdn-...|
|  Grand Theft Auto V| 35155163|     271590|https://steamcdn-...|
|Counter-Strike: S...| 29459482|        240|https://steamcdn-...|
|            Warframe| 28659453|     230410|https://steamcdn-...|
|       Left 4 Dead 2| 21343026|        550|https://steamcdn-...|
|            PAYDAY 2| 21032415|     218620|https://steamcdn-...|
|      Counter-Strike| 19126275|         10|https://steamcdn-...|
|            Terraria| 18232159|     105600|https://steamcdn-...|
|The Elder Scrolls...| 18182393|      72850|https://steamcdn-...|
+--------------------+---------+-----------+--------------------+



### 친구 리스트 처리

In [ ]:
# 친구 리스트 데이터 등록

sample_user = '76561197960434622'

df_user_friend_list = sc.read.json(user_friend_list)
df_user_friend_list.printSchema()
df_user_friend_list.registerTempTable('friend_list')


df_friend_list = sc.sql("SELECT friends['steamid'] AS steamid FROM \
            (SELECT EXPLODE(friends) AS friends FROM friend_list WHERE steamid = %s) a"%sample_user)
df_friend_list.show(10)
df_friend_list.registerTempTable('user_friend_list')

root
 |-- friends: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- friend_since: long (nullable = true)
 |    |    |-- relationship: string (nullable = true)
 |    |    |-- steamid: string (nullable = true)
 |-- steamid: string (nullable = true)

+-----------------+
|          steamid|
+-----------------+
|76561197960265738|
|76561197960265740|
|76561197960265743|
|76561197960265754|
|76561197960265838|
|76561197960269198|
|76561197960275076|
|76561197960276281|
|76561197960327485|
|76561197960381818|
+-----------------+
only showing top 10 rows



In [ ]:
# 친구의 게임 플레이 시간을 등록
sc.sql("SELECT game_id, SUM(playtime_forever) AS play_time FROM \
            (SELECT games['appid'] AS game_id, games['playtime_forever'] AS playtime_forever FROM \
            (SELECT a.steamid, EXPLODE(b.games) AS games \
            FROM user_friend_list a, user_owned_games b WHERE a.steamid = b.steamid) c) d \
            GROUP BY game_id ORDER BY play_time DESC LIMIT 10")\
.registerTempTable('temp_local_popular_games')

df_global_popular_games = sc.sql("SELECT DISTINCT b.name AS game_name, a.play_time FROM \
                                        temp_local_popular_games a, game_detail b WHERE a.game_id = b.steam_appid")
df_global_popular_games.show()

+--------------------+---------+
|           game_name|play_time|
+--------------------+---------+
|Counter-Strike: G...|  1011736|
|     Elite Dangerous|   158316|
|         Left 4 Dead|   113271|
|       Left 4 Dead 2|   148037|
|       Borderlands 2|   120164|
|Counter-Strike: S...|   243005|
|        Just Cause 2|   121389|
|      Clicker Heroes|   805415|
|            Warframe|   344961|
+--------------------+---------+



## Collaboratie Filtering

CF implementation using the tequenics that aims to fill empty entries in the matrix

Spark mllib uses Alternating Least Square Algorithms to learn these factors

reference : https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

### 유저의 최근 플레이 게임을 기반으로 데이터 처리

In [ ]:
df_user_recent_games = sc.read.json(user_recent_games)
df_user_recent_games.printSchema()
df_user_recent_games.registerTempTable("user_recent_games")
df_valid_user_recent_games = sc.sql("SELECT * FROM user_recent_games where total_count != 0")
df_valid_user_recent_games.show(1)

root
 |-- games: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appid: long (nullable = true)
 |    |    |-- img_icon_url: string (nullable = true)
 |    |    |-- img_logo_url: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- playtime_2weeks: long (nullable = true)
 |    |    |-- playtime_forever: long (nullable = true)
 |    |    |-- playtime_linux_forever: long (nullable = true)
 |    |    |-- playtime_mac_forever: long (nullable = true)
 |    |    |-- playtime_windows_forever: long (nullable = true)
 |-- steamid: string (nullable = true)
 |-- total_count: string (nullable = true)

+--------------------+-----------------+-----------+
|               games|          steamid|total_count|
+--------------------+-----------------+-----------+
|[[427520, 267f5a8...|76561197960434622|          7|
+--------------------+-----------------+-----------+
only showing top 1 row



In [ ]:
df_user_idx = sc.read.json(user_idx)
df_user_idx.printSchema()
df_user_idx.registerTempTable('user_idx')
df_valid_user_recent_games = sc.sql("SELECT b.user_idx, a.games FROM user_recent_games a \
                                            JOIN user_idx b ON b.user_id = a.steamid WHERE a.total_count != 0")
df_valid_user_recent_games.printSchema()
df_valid_user_recent_games.show(10)

root
 |-- user_id: string (nullable = true)
 |-- user_idx: long (nullable = true)

root
 |-- user_idx: long (nullable = true)
 |-- games: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- appid: long (nullable = true)
 |    |    |-- img_icon_url: string (nullable = true)
 |    |    |-- img_logo_url: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- playtime_2weeks: long (nullable = true)
 |    |    |-- playtime_forever: long (nullable = true)
 |    |    |-- playtime_linux_forever: long (nullable = true)
 |    |    |-- playtime_mac_forever: long (nullable = true)
 |    |    |-- playtime_windows_forever: long (nullable = true)

+--------+--------------------+
|user_idx|               games|
+--------+--------------------+
|       0|[[427520, 267f5a8...|
|       6|[[255710, 6cf7b10...|
|       7|[[1151640, 08a1b6...|
|      13|[[300570, 7566ab0...|
|      15|[[550, 7d5a243f95...|
|      20|[[440, e3f595a925...|
|    

### 최근 플레이 타임을 기반으로 Matrix를 생성, 이후 Spark의 mllib ALS 를 사용하여 트레이닝

In [ ]:
# spark rdd
# 플레이 타임을 flat으로 매핑
# 플레이 타임이 0인 게임들은 제외함
training_rdd = df_valid_user_recent_games.rdd.flatMapValues(lambda x : x).map(lambda x_y : (x_y[0], x_y[1].appid, x_y[1].playtime_forever)).filter(lambda x_y_z : x_y_z[2] > 0)
training_rdd.collect()

[(0, 427520, 27516),
 (0, 296490, 434),
 (0, 322330, 161),
 (0, 1295510, 64),
 (0, 1127400, 244),
 (0, 271590, 2051),
 (0, 12100, 41),
 (6, 255710, 2972),
 (7, 1151640, 3331),
 (7, 1262580, 257),
 (13, 300570, 20332),
 (13, 965200, 72345),
 (13, 418950, 125),
 (13, 376520, 95),
 (13, 372210, 90),
 (13, 311850, 90),
 (13, 746140, 214),
 (15, 550, 144049),
 (15, 386360, 15102),
 (15, 228380, 31607),
 (15, 563560, 21455),
 (15, 252950, 6451),
 (15, 312530, 108),
 (20, 440, 122536),
 (23, 108800, 665),
 (23, 714120, 335),
 (23, 320240, 159),
 (23, 960090, 85),
 (23, 550, 878),
 (23, 1150950, 690),
 (2571, 359550, 156842),
 (2571, 730, 152732),
 (2571, 945360, 258),
 (2571, 674940, 114),
 (2571, 24790, 515),
 (26, 359550, 156842),
 (26, 730, 152732),
 (26, 945360, 258),
 (26, 674940, 114),
 (26, 24790, 515),
 (27, 945360, 2276),
 (27, 620, 3098),
 (27, 728880, 302),
 (27, 400, 169),
 (27, 252950, 11752),
 (27, 437920, 351),
 (27, 1016920, 212),
 (28, 730, 404015),
 (28, 406290, 11916),
 (28

In [ ]:
als_model = ALS.trainImplicit(training_rdd, 10)

In [ ]:
result_rating = als_model.recommendProducts(0,10)
print(result_rating)
try_df_result=sc.createDataFrame(result_rating)
print(try_df_result.sort(desc("rating")).show())

[Rating(user=0, product=4000, rating=1.3899624905044765), Rating(user=0, product=427520, rating=0.9194556935749931), Rating(user=0, product=431960, rating=0.9102788608466068), Rating(user=0, product=1085660, rating=0.8106474151898326), Rating(user=0, product=252490, rating=0.7728388815917631), Rating(user=0, product=238960, rating=0.7595273440469079), Rating(user=0, product=440, rating=0.7221293494909724), Rating(user=0, product=550, rating=0.7179422485119016), Rating(user=0, product=1840, rating=0.7087060525654937), Rating(user=0, product=240, rating=0.6436598867218222)]
+----+-------+------------------+
|user|product|            rating|
+----+-------+------------------+
|   0|   4000|1.3899624905044765|
|   0| 427520|0.9194556935749931|
|   0| 431960|0.9102788608466068|
|   0|1085660|0.8106474151898326|
|   0| 252490|0.7728388815917631|
|   0| 238960|0.7595273440469079|
|   0|    440|0.7221293494909724|
|   0|    550|0.7179422485119016|
|   0|   1840|0.7087060525654937|
|   0|    240

## 결과물 저장

### 추천 게임 목록을 json형식으로 저장

In [ ]:
# 유저가 플레이한 게임이 없다면 결과값 누락이 있을 수 있음. (매트릭스 기반이기 때문에)
# 따라서 이런 콜드 스타트의 경우 글로벌 인기 게임을 추천

with open(recommended, 'w') as output_file:
    for user_idx in range(0, df_user_idx.count()):
        try:
            lst_recommended = [i.product for i in als_model.recommendProducts(user_idx, 10)]
            rank = 1
            for app_id in lst_recommended:
                dict_recommended = {'user_idx': user_idx, 'game_id': app_id, 'rank': rank}
                json.dump(dict_recommended, output_file)
                output_file.write('\n')
                rank += 1
        except:
            pass

### 최종 결과물 저장

In [ ]:
df_recommend_result = sc.read.json(recommended)
df_recommend_result.show(20)

+-------+----+--------+
|game_id|rank|user_idx|
+-------+----+--------+
|   4000|   1|       0|
| 427520|   2|       0|
| 431960|   3|       0|
|1085660|   4|       0|
| 252490|   5|       0|
| 238960|   6|       0|
|    440|   7|       0|
|    550|   8|       0|
|   1840|   9|       0|
|    240|  10|       0|
| 255710|   1|       6|
| 292030|   2|       6|
|1145360|   3|       6|
| 230410|   4|       6|
|  39210|   5|       6|
| 306130|   6|       6|
|1147690|   7|       6|
|    550|   8|       6|
| 270880|   9|       6|
|1046930|  10|       6|
+-------+----+--------+
only showing top 20 rows



In [ ]:
df_recommend_result.registerTempTable('recommend_result')
df_final_recommend_result = sc.sql("SELECT DISTINCT b.user_id, a.rank, c.name, c.genres, c.short_description, c.steam_appid \
                                        FROM recommend_result a, user_idx b, game_detail c \
                                        WHERE a.user_idx = b.user_idx AND a.game_id = c.steam_appid \
                                        ORDER BY b.user_id, a.rank") 
df_final_recommend_result.show(20)

+-----------------+----+--------------------+--------------------+--------------------+-----------+
|          user_id|rank|                name|              genres|   short_description|steam_appid|
+-----------------+----+--------------------+--------------------+--------------------+-----------+
|76561197960265754|   1|  Grand Theft Auto V|[[Action, 1], [Ad...|Grand Theft Auto ...|     271590|
|76561197960265754|   2|           Destiny 2|[[Action, 1], [Ad...|Destiny 2 is an a...|    1085660|
|76561197960265754|   4|Halo: The Master ...|       [[Action, 1]]|The Master Chief’...|     976730|
|76561197960265754|   5|            Among Us|       [[Casual, 4]]|An online and loc...|     945360|
|76561197960265754|   6|       7 Days to Die|[[Action, 1], [Ad...|7 Days to Die is ...|     251570|
|76561197960265754|   7|Counter-Strike: G...|[[Action, 1], [Fr...|Counter-Strike: G...|        730|
|76561197960265754|   8|            Warframe|[[Action, 1], [Fr...|Warframe is a coo...|     230410|


In [ ]:
# 최종 추천값 전달
df_final_recommend_result.write.save(final_recommended, format="json")